In [1]:
from pprint import pprint
import requests
import nltk
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [3]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

In [9]:
def get_headers(soup):
    news = []
    headers_container = soup.find_all("h2", {'class': 'titulo'})
    for header in headers_container:
        news.append(header.text)
        
    print(news)    
    return pd.DataFrame(news)

In [16]:
def clean_accent(newsdf):
    return newsdf.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')


In [19]:
soup = get_soup(MADRID_URL)
dfnews_raw = get_headers(soup) #return dataframe

dfnews = clean_accent(dfnews_raw)
dfnews.head(40)


['Un hombre sufre varios traumatismos graves al caer con su bicicleta en la carretera', 'Desarticulado un grupo criminal que se dedicaba al robo de coches para su posterior venta', 'Herido muy grave un hombre invidente y sordomudo atropellado por una moto', "La sentencia del Supremo a 'La Manada' reaviva el debate sobre la reforma del Código Penal", 'Localizado el argentino de 72 años desaparecido en Barajas', "Más de 130 organizaciones feministas volverán a entonar el 'Yo sí te creo'", 'La fiscal defiende que La Manada cometió una violación continuada', 'Inhabilitada una pediatra que vinculaba las vacunaciones con el autismo', 'El Supremo escucha los recursos contra la sentencia de La Manada', 'Un chico de 16 años acuchilla a su hermano y un amigo en San Blas', 'El PP cesa al director general de la Policía sin tener sustituto', 'Condenado a cuatro años el asesino de Olga Sangrador por abusos a otra menor en Madrid', 'Detenidos 15 estafadores que se hacían pasar por técnicos de gas y e

,0
0,Un hombre sufre varios traumatismos graves al ...
1,Desarticulado un grupo criminal que se dedicab...
2,Herido muy grave un hombre invidente y sordomu...
3,La sentencia del Supremo a 'La Manada' reaviva...
4,Localizado el argentino de 72 años desaparecid...
5,Más de 130 organizaciones feministas volverán ...
6,La fiscal defiende que La Manada cometió una v...
7,Inhabilitada una pediatra que vinculaba las va...
8,El Supremo escucha los recursos contra la sent...
9,Un chico de 16 años acuchilla a su hermano y u...
